In [1]:
!pip install git+https://github.com/kmkurn/pytorch-crf.git

  Cloning https://github.com/kmkurn/pytorch-crf.git to /tmp/pip-req-build-ay00quy9
  Running command git clone --filter=blob:none --quiet https://github.com/kmkurn/pytorch-crf.git /tmp/pip-req-build-ay00quy9
  Resolved https://github.com/kmkurn/pytorch-crf.git to commit 623e3402d00a2728e99d6e8486010d67c754267b
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-crf: filename=pytorch_crf-0.7.2-py3-none-any.whl size=6410 sha256=33d5921fd09856edf8ee77b57bf8a00271077829d301dc8d2fb9f2e645e3886d
  Stored in directory: /tmp/pip-ephem-wheel-cache-6m1nivts/wheels/fd/83/cc/f11543939f8911b8dcff86e2bd54423e21f779d0938958cc7f
Successfully built pytorch-crf


In [3]:
# =================================================================
# CELL 0: CÀI ĐẶT VÀ KHỞI TẠO VnCoreNLP
# =================================================================
!pip install vncorenlp -q

from vncorenlp import VnCoreNLP
import os


# 1. Tạo cấu trúc thư mục chuẩn
!mkdir -p vncorenlp/models/wordsegmenter

# 2. Tải các file cần thiết
print("Đang tải các file cần thiết cho VnCoreNLP...")
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar -q
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab -q
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr -q
print("Tải xong!")

# 3. Di chuyển các file vào đúng vị trí
!mv VnCoreNLP-1.2.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

# 4. Khởi tạo VnCoreNLP (sẽ rất nhanh và không bị lỗi kết nối)
print("\nĐang khởi tạo VnCoreNLP...")
# Bây giờ, nó sẽ load file trực tiếp thay vì chạy server
vncorenlp_segmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.2.jar", annotators="wseg", max_heap_size='-Xmx500m')
print("VnCoreNLP đã sẵn sàng.")


# --- Các phần còn lại của code setup ---
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import torch.nn as nn
from torchcrf import CRF # Giả sử bạn đã cài đặt torchcrf thành công
from google.colab import drive

# Kết nối Drive
drive.mount('/content/drive')

# Ví dụ câu đầu vào
raw_sentence = "Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024"

# Thử tách từ để kiểm tra
word_segmented_list = vncorenlp_segmenter.tokenize(raw_sentence)
print(f"\nCâu gốc: {raw_sentence}")
print(f"Câu đã tách từ: {' '.join(word_segmented_list[0])}")

Đang tải các file cần thiết cho VnCoreNLP...
Tải xong!

Đang khởi tạo VnCoreNLP...
VnCoreNLP đã sẵn sàng.
Mounted at /content/drive

Câu gốc: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024
Câu đã tách từ: Tăng_trưởng doanh_thu mảng khách_hàng cá_nhân đạt 20% trong quý 4 năm 2024


In [12]:
# =================================================================
# CELL 1: TEST MÔ HÌNH PhoBERT - SYLLABLE
# =================================================================
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import torch

# Đường dẫn tới mô hình đã lưu
MODEL_PATH = "/content/drive/MyDrive/kpi_ner_outputs_phobert/phobert-kpi-ner-syllable/final_model_syllable"

# Tải tokenizer và model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)

# Tạo pipeline NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# ==================== THAY ĐỔI TẠI ĐÂY ====================
# 1. Thay thế một câu bằng một danh sách các câu
raw_sentences_list = [
    "Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024",
    "Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay.",
    "Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này.",
    "Bộ phận Marketing cần duy trì lượng truy cập trang web ở mức 10,000 lượt/ngày trong suốt quý 2.",
    "Tăng doanh số bán hàng thêm 15% so với cùng kỳ năm ngoái.",
    "Anh Tuấn cần đạt được chỉ số NPS trên 70 điểm trong vòng 2 tháng.",
    "Mục tiêu: giảm 10% chi phí vận hành trong quý 4."
    # Thêm các câu khác của bạn vào đây
]

# 2. Lặp qua danh sách câu để dự đoán và in kết quả
for i, sentence in enumerate(raw_sentences_list):
    # Tách câu thành các âm tiết
    syllables = sentence.split()
    # Nối lại bằng khoảng trắng để pipeline xử lý đúng
    syllable_sentence = " ".join(syllables)

    print(f"\n--- Câu {i+1}: {sentence} ---")
    print(f"Input cho model: {syllable_sentence}")

    predictions = ner_pipeline(syllable_sentence)

    print("Kết quả dự đoán:")
    if not predictions:
        print("  (Không tìm thấy thực thể nào)")
    else:
        for entity in predictions:
            print(f"  Thực thể: {entity['word']}")
            print(f"    - Nhãn: {entity['entity_group']}")

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



--- Câu 1: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024 ---
Input cho model: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024
Kết quả dự đoán:
  Thực thể: Tăng trưởng doanh thu mảng khách hàng cá nhân
    - Nhãn: KPI
  Thực thể: 20%
    - Nhãn: TARGET
  Thực thể: quý 4 năm 2024
    - Nhãn: TIME_FRAME

--- Câu 2: Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay. ---
Input cho model: Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay.
Kết quả dự đoán:
  Thực thể: Anh Nam
    - Nhãn: OWNER
  Thực thể: tăng
    - Nhãn: DIRECTION
  Thực thể: tỷ lệ phản hồi khách hàng
    - Nhãn: KPI
  Thực thể: 95%
    - Nhãn: TARGET
  Thực thể: quý 3 năm nay
    - Nhãn: TIME_FRAME

--- Câu 3: Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này. ---
Input cho model: Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này.
Kết quả dự đoán:
  Thực thể: Chị La

In [13]:
# =================================================================
# CELL 2: TEST MÔ HÌNH PhoBERT - WORD
# =================================================================
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import torch

# Đường dẫn tới mô hình đã lưu
MODEL_PATH = "/content/drive/MyDrive/kpi_ner_outputs_phobert/phobert-kpi-ner-word/final_model_word"

# Tải tokenizer và model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)

# Tạo pipeline NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


# ==================== THAY ĐỔI TẠI ĐÂY ====================
# 1. Tạo một danh sách các câu cần dự đoán
# Giả sử biến vncorenlp_segmenter đã được khởi tạo ở ô trước
raw_sentences_list = [
    "Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024",
    "Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay.",
    "Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này.",
    "Bộ phận Marketing cần duy trì lượng truy cập trang web ở mức 10,000 lượt/ngày trong suốt quý 2.",
    "Tăng doanh số bán hàng thêm 15% so với cùng kỳ năm ngoái.",
    "Anh Tuấn cần đạt được chỉ số NPS trên 70 điểm trong vòng 2 tháng.",
    "Mục tiêu: giảm 10% chi phí vận hành trong quý 4."
    # Thêm các câu khác của bạn vào đây
]

# 2. Lặp qua danh sách câu để dự đoán và in kết quả
for i, sentence in enumerate(raw_sentences_list):
    # Tách từ bằng VnCoreNLP
    word_segmented_list = vncorenlp_segmenter.tokenize(sentence)
    # Nối lại các từ đã tách thành một câu
    word_segmented_sentence = " ".join(word_segmented_list[0])

    print(f"\n--- Câu {i+1}: {sentence} ---")
    print(f"Input cho model: {word_segmented_sentence}")

    predictions = ner_pipeline(word_segmented_sentence)

    print("Kết quả dự đoán:")
    if not predictions:
        print("  (Không tìm thấy thực thể nào)")
    else:
        for entity in predictions:
            # Lưu ý: pipeline sẽ tự xử lý việc nối lại các token có dấu _
            # thành một thực thể hoàn chỉnh, ví dụ "khách_hàng" -> "khách hàng"
            print(f"  Thực thể: {entity['word']}")
            print(f"    - Nhãn: {entity['entity_group']}")

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



--- Câu 1: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024 ---
Input cho model: Tăng_trưởng doanh_thu mảng khách_hàng cá_nhân đạt 20% trong quý 4 năm 2024
Kết quả dự đoán:
  Thực thể: Tăng_trưởng doanh_thu mảng khách_hàng cá_nhân
    - Nhãn: KPI
  Thực thể: 20%
    - Nhãn: TARGET
  Thực thể: quý 4 năm 2024
    - Nhãn: TIME_FRAME

--- Câu 2: Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay. ---
Input cho model: Anh Nam cần tăng tỷ_lệ phản_hồi khách_hàng lên 95% trong quý 3 năm nay .
Kết quả dự đoán:
  Thực thể: Anh Nam
    - Nhãn: OWNER
  Thực thể: tăng
    - Nhãn: DIRECTION
  Thực thể: tỷ_lệ phản_hồi khách_hàng
    - Nhãn: KPI
  Thực thể: 95%
    - Nhãn: TARGET
  Thực thể: quý 3 năm nay
    - Nhãn: TIME_FRAME

--- Câu 3: Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này. ---
Input cho model: Chị Lan phải giảm thời_gian xử_lý đơn hàng xuống dưới 2 giờ trước cuối tháng này .
Kết quả dự đoán:
  Thực thể: Chị 

In [14]:
# =================================================================
# TEST MÔ HÌNH XLM-R - SYLLABLE
# =================================================================
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import torch

# Đường dẫn tới mô hình đã lưu
MODEL_PATH = "/content/drive/MyDrive/kpi_ner_outputs_xlmr/xlm-roberta-base-kpi-ner-syllable/final_model_syllable"

# Tải tokenizer và model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)

# Tạo pipeline NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# ==================== DANH SÁCH CÂU ĐẦU VÀO ====================
raw_sentences_list = [
    "Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024",
    "Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay.",
    "Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này.",
    "Bộ phận Marketing cần duy trì lượng truy cập trang web ở mức 10,000 lượt/ngày trong suốt quý 2.",
    "Tăng doanh số bán hàng thêm 15% so với cùng kỳ năm ngoái.",
    "Anh Tuấn cần đạt được chỉ số NPS trên 70 điểm trong vòng 2 tháng.",
    "Mục tiêu: giảm 10% chi phí vận hành trong quý 4."
    # Thêm câu khác tại đây nếu cần
]

# ==================== CHẠY DỰ ĐOÁN ====================
for i, sentence in enumerate(raw_sentences_list):
    # Tách câu thành các âm tiết (dựa trên khoảng trắng)
    syllables = sentence.split()
    syllable_sentence = " ".join(syllables)

    print(f"\n--- Câu {i+1}: {sentence} ---")
    print(f"Input cho model: {syllable_sentence}")

    # Dự đoán NER
    predictions = ner_pipeline(syllable_sentence)

    # Hiển thị kết quả
    print("Kết quả dự đoán:")
    if not predictions:
        print("  (Không tìm thấy thực thể nào)")
    else:
        for entity in predictions:
            print(f"  Thực thể: {entity['word']}")
            print(f"    - Nhãn: {entity['entity_group']}")


Device set to use cpu



--- Câu 1: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024 ---
Input cho model: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024
Kết quả dự đoán:
  Thực thể: Tăng trưởng doanh thu mảng khách hàng cá nhân
    - Nhãn: KPI
  Thực thể: 20%
    - Nhãn: TARGET
  Thực thể: quý 4 năm 2024
    - Nhãn: TIME_FRAME

--- Câu 2: Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay. ---
Input cho model: Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay.
Kết quả dự đoán:
  Thực thể: Anh Nam
    - Nhãn: OWNER
  Thực thể: tăng
    - Nhãn: DIRECTION
  Thực thể: tỷ lệ phản hồi khách hàng
    - Nhãn: KPI
  Thực thể: 95%
    - Nhãn: TARGET
  Thực thể: quý 3 năm nay
    - Nhãn: TIME_FRAME

--- Câu 3: Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này. ---
Input cho model: Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này.
Kết quả dự đoán:
  Thực thể: Chị La

In [15]:
# =================================================================
# TEST MÔ HÌNH XLM-R - WORD
# =================================================================
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import torch

# Đường dẫn tới mô hình đã lưu
MODEL_PATH = "/content/drive/MyDrive/kpi_ner_outputs_xlmr/xlm-roberta-base-kpi-ner-word/final_model_word"

# Tải tokenizer và model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)

# Tạo pipeline NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# ==================== DANH SÁCH CÂU ĐẦU VÀO ====================
raw_sentences_list = [
    "Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024",
    "Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay.",
    "Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này.",
    "Bộ phận Marketing cần duy trì lượng truy cập trang web ở mức 10,000 lượt/ngày trong suốt quý 2.",
    "Tăng doanh số bán hàng thêm 15% so với cùng kỳ năm ngoái.",
    "Anh Tuấn cần đạt được chỉ số NPS trên 70 điểm trong vòng 2 tháng.",
    "Mục tiêu: giảm 10% chi phí vận hành trong quý 4."
]

# ==================== CHẠY DỰ ĐOÁN ====================
for i, sentence in enumerate(raw_sentences_list):
    # Tách từ bằng VnCoreNLP
    word_segmented_list = vncorenlp_segmenter.tokenize(sentence)
    word_segmented_sentence = " ".join(word_segmented_list[0])

    print(f"\n--- Câu {i+1}: {sentence} ---")
    print(f"Input cho model (tách từ): {word_segmented_sentence}")

    # Dự đoán NER
    predictions = ner_pipeline(word_segmented_sentence)

    # Hiển thị kết quả
    print("Kết quả dự đoán:")
    if not predictions:
        print("  (Không tìm thấy thực thể nào)")
    else:
        for entity in predictions:
            print(f"  Thực thể: {entity['word']}")
            print(f"    - Nhãn: {entity['entity_group']}")


Device set to use cpu



--- Câu 1: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024 ---
Input cho model (tách từ): Tăng_trưởng doanh_thu mảng khách_hàng cá_nhân đạt 20% trong quý 4 năm 2024
Kết quả dự đoán:
  Thực thể: Tăng_trưởng doanh_thu mảng khách_hàng cá_nhân
    - Nhãn: KPI
  Thực thể: 20%
    - Nhãn: TARGET
  Thực thể: quý 4 năm 2024
    - Nhãn: TIME_FRAME

--- Câu 2: Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay. ---
Input cho model (tách từ): Anh Nam cần tăng tỷ_lệ phản_hồi khách_hàng lên 95% trong quý 3 năm nay .
Kết quả dự đoán:
  Thực thể: Anh Nam
    - Nhãn: OWNER
  Thực thể: tăng
    - Nhãn: DIRECTION
  Thực thể: tỷ_lệ phản_hồi khách_hàng
    - Nhãn: KPI
  Thực thể: 95%
    - Nhãn: TARGET
  Thực thể: quý 3 năm nay
    - Nhãn: TIME_FRAME

--- Câu 3: Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này. ---
Input cho model (tách từ): Chị Lan phải giảm thời_gian xử_lý đơn hàng xuống dưới 2 giờ trước cuối tháng này .
Kết

In [16]:
# =================================================================
# NOTEBOOK DỰ ĐOÁN SỬ DỤNG MODEL BiLSTM-CNN-CRF ĐÃ HUẤN LUYỆN syllable
# =================================================================

# --- BƯỚC 1: CÀI ĐẶT CÁC THƯ VIỆN CẦN THIẾT ---
# Chỉ cài đặt những gì cần cho việc dự đoán
!pip install git+https://github.com/kmkurn/pytorch-crf.git
!pip install vncorenlp -q # Nếu bạn muốn test mô hình word-level

from google.colab import drive
import torch
import torch.nn as nn
from torchcrf import CRF
import os
import json
from vncorenlp import VnCoreNLP

# --- BƯỚC 2: KẾT NỐI DRIVE VÀ ĐỊNH NGHĨA CÁC THÀNH PHẦN ---

# Kết nối Google Drive
drive.mount('/content/drive')

# Xác định thiết bị (sẽ tự động dùng GPU nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

# Định nghĩa lại kiến trúc model (phải giống hệt lúc huấn luyện)
class BiLSTM_CNN_CRF(nn.Module):
    def __init__(self, vocab_size, char_vocab_size, tag_to_idx,
                 word_emb_dim, char_emb_dim, char_cnn_out_channels,
                 hidden_dim, num_lstm_layers, dropout):
        super(BiLSTM_CNN_CRF, self).__init__()
        self.tagset_size = len(tag_to_idx)
        self.word_embedding = nn.Embedding(vocab_size, word_emb_dim, padding_idx=0)
        self.char_embedding = nn.Embedding(char_vocab_size, char_emb_dim, padding_idx=0)
        self.char_cnn = nn.Conv1d(char_emb_dim, char_cnn_out_channels, kernel_size=3, padding=1)
        self.bilstm = nn.LSTM(word_emb_dim + char_cnn_out_channels, hidden_dim // 2,
                              num_layers=num_lstm_layers, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.crf = CRF(self.tagset_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, words, chars, tags=None, mask=None):
        word_embeds = self.word_embedding(words)
        batch_size, seq_len, word_len = chars.shape
        chars = chars.view(batch_size * seq_len, word_len)
        char_embeds = self.char_embedding(chars).transpose(1, 2)
        char_cnn_out = self.char_cnn(char_embeds)
        char_features = torch.max(char_cnn_out, 2)[0].view(batch_size, seq_len, -1)
        combined_embeds = self.dropout(torch.cat((word_embeds, char_features), 2))
        lstm_out, _ = self.bilstm(combined_embeds)
        emissions = self.hidden2tag(self.dropout(lstm_out))
        if tags is not None:
            return -self.crf(emissions, tags, mask=mask.byte(), reduction='mean')
        else:
            return self.crf.decode(emissions, mask=mask.byte())

# Định nghĩa hàm dự đoán
def predict_bilstm(sentence, model, word_to_idx, char_to_idx, idx_to_tag):
    model.eval()
    tokens = sentence.split()
    if not tokens: return []

    word_indices = [word_to_idx.get(w, word_to_idx["<UNK>"]) for w in tokens]
    char_indices_per_word = [[char_to_idx.get(c, char_to_idx["<UNK>"]) for c in w] for w in tokens]

    max_word_len = max([len(c) for c in char_indices_per_word if c] or [1])
    padded_chars = []
    for chars in char_indices_per_word:
        padded = torch.zeros(max_word_len, dtype=torch.long)
        if chars: padded[:len(chars)] = torch.tensor(chars)
        padded_chars.append(padded)

    words_tensor = torch.tensor([word_indices], dtype=torch.long).to(device)
    chars_tensor = torch.stack(padded_chars).unsqueeze(0).to(device)
    mask = torch.ones_like(words_tensor, dtype=torch.bool).to(device)

    with torch.no_grad():
        predicted_tag_indices = model(words_tensor, chars_tensor, mask=mask)[0]

    predicted_tags = [idx_to_tag.get(i, 'O') for i in predicted_tag_indices]

    entities, current_entity, current_tag = [], "", ""
    for token, tag in zip(tokens, predicted_tags):
        token_to_add = token.replace("_", " ")
        if tag.startswith("B-"):
            if current_entity: entities.append({"entity": current_entity.strip(), "tag": current_tag})
            current_entity, current_tag = token_to_add + " ", tag[2:]
        elif tag.startswith("I-") and current_tag and tag[2:] == current_tag:
            current_entity += token_to_add + " "
        else:
            if current_entity: entities.append({"entity": current_entity.strip(), "tag": current_tag})
            current_entity, current_tag = "", ""
    if current_entity: entities.append({"entity": current_entity.strip(), "tag": current_tag})

    return entities

# --- BƯỚC 3: TẢI MODEL VÀ TỪ ĐIỂN ---

# Chọn model bạn muốn test (syllable hoặc word)
DATA_TYPE = "syllable" # <-- THAY ĐỔI "syllable" hoặc "word" ở đây

MODEL_DIR = f"/content/drive/MyDrive/kpi_ner_outputs_bilstm_with_vocabs/bilstm-cnn-crf-{DATA_TYPE}"
print(f"Đang tải model và từ điển cho: {DATA_TYPE}")

# Tải từ điển từ file vocabs.json
vocab_path = os.path.join(MODEL_DIR, "vocabs.json")
try:
    with open(vocab_path, "r", encoding="utf-8") as f:
        vocabs = json.load(f)
    word_to_idx = vocabs["word_to_idx"]
    char_to_idx = vocabs["char_to_idx"]
    tag_to_idx = vocabs["tag_to_idx"]
    idx_to_tag = {int(i): t for t, i in tag_to_idx.items()}
    print("Đã tải từ điển thành công.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file 'vocabs.json' tại '{MODEL_DIR}'.")
    raise

# Khởi tạo mô hình với đúng cấu trúc từ từ điển đã tải
model_to_test = BiLSTM_CNN_CRF(
    vocab_size=len(word_to_idx), char_vocab_size=len(char_to_idx), tag_to_idx=tag_to_idx,
    word_emb_dim=100, char_emb_dim=30, char_cnn_out_channels=30, hidden_dim=256, num_lstm_layers=2, dropout=0.5
).to(device)

# Tải trọng số đã huấn luyện
model_weights_path = os.path.join(MODEL_DIR, "best_model.pt")
model_to_test.load_state_dict(torch.load(model_weights_path, map_location=device))
print(f"Đã tải trọng số model thành công.")

# Khởi tạo VnCoreNLP nếu cần test mô hình word
vncorenlp_segmenter = None
if DATA_TYPE == "word":
    print("\nKhởi tạo VnCoreNLP cho mô hình word-level...")
    # Dùng phương pháp cài đặt ổn định
    !mkdir -p vncorenlp/models/wordsegmenter
    if not os.path.exists('vncorenlp/VnCoreNLP-1.2.jar'):
        !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar -q -O vncorenlp/VnCoreNLP-1.2.jar
        !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab -q -O vncorenlp/models/wordsegmenter/vi-vocab
        !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr -q -O vncorenlp/models/wordsegmenter/wordsegmenter.rdr
    vncorenlp_segmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.2.jar", annotators="wseg", max_heap_size='-Xmx500m')
    print("VnCoreNLP sẵn sàng.")

# --- BƯỚC 4: THỰC HIỆN DỰ ĐOÁN ---

# Danh sách các câu bạn muốn test
raw_sentences_list = [
    "Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024",
    "Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay.",
    "Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này.",
    "Bộ phận Marketing cần duy trì lượng truy cập trang web ở mức 10,000 lượt/ngày trong suốt quý 2.",
    "Tăng doanh số bán hàng thêm 15% so với cùng kỳ năm ngoái.",
    "Anh Tuấn cần đạt được chỉ số NPS trên 70 điểm trong vòng 2 tháng.",
    "Mục tiêu: giảm 10% chi phí vận hành trong quý 4."
]

print("\n" + "="*20 + " BẮT ĐẦU DỰ ĐOÁN " + "="*20)
for i, sentence in enumerate(raw_sentences_list):
    print(f"\n--- Câu {i+1}: {sentence} ---")

    # Tiền xử lý câu đầu vào cho phù hợp với model
    input_sentence = ""
    if DATA_TYPE == "syllable":
        input_sentence = " ".join(sentence.split())
    elif DATA_TYPE == "word":
        if vncorenlp_segmenter:
            tokenized = vncorenlp_segmenter.tokenize(sentence)
            input_sentence = " ".join(tokenized[0])
        else:
            print("Lỗi: VnCoreNLP chưa được khởi tạo để tách từ.")
            continue

    print(f"Input cho model ({DATA_TYPE}-level): {input_sentence}")

    # Gọi hàm dự đoán
    predictions = predict_bilstm(input_sentence, model_to_test, word_to_idx, char_to_idx, idx_to_tag)

    print("Các thực thể được tìm thấy:")
    if not predictions:
        print("  (Không tìm thấy thực thể nào)")
    else:
        for entity in predictions:
            print(f"  - Thực thể: '{entity['entity']}'")
            print(f"    - Nhãn: {entity['tag']}")

print("\n" + "="*20 + " HOÀN TẤT " + "="*20)

  Cloning https://github.com/kmkurn/pytorch-crf.git to /tmp/pip-req-build-fwhk84uz
  Running command git clone --filter=blob:none --quiet https://github.com/kmkurn/pytorch-crf.git /tmp/pip-req-build-fwhk84uz
  Resolved https://github.com/kmkurn/pytorch-crf.git to commit 623e3402d00a2728e99d6e8486010d67c754267b
  Preparing metadata (setup.py) ... done
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Đang sử dụng thiết bị: cpu
Đang tải model và từ điển cho: syllable
Đã tải từ điển thành công.
Đã tải trọng số model thành công.

==================== BẮT ĐẦU DỰ ĐOÁN ====================

--- Câu 1: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024 ---
Input cho model (syllable-level): Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024
Các thực thể được tìm thấy:
  - Thực thể: 'Tăng trưởng doanh thu mảng khách hàng cá nhân'
    - Nhãn: KPI
  - Thực thể: '20%'
    -

In [17]:
# =================================================================
# NOTEBOOK DỰ ĐOÁN SỬ DỤNG MODEL BiLSTM-CNN-CRF ĐÃ HUẤN LUYỆN word
# =================================================================

# --- BƯỚC 1: CÀI ĐẶT CÁC THƯ VIỆN CẦN THIẾT ---
# Chỉ cài đặt những gì cần cho việc dự đoán
!pip install git+https://github.com/kmkurn/pytorch-crf.git
!pip install vncorenlp -q # Nếu bạn muốn test mô hình word-level

from google.colab import drive
import torch
import torch.nn as nn
from torchcrf import CRF
import os
import json
from vncorenlp import VnCoreNLP

# --- BƯỚC 2: KẾT NỐI DRIVE VÀ ĐỊNH NGHĨA CÁC THÀNH PHẦN ---

# Kết nối Google Drive
drive.mount('/content/drive')

# Xác định thiết bị (sẽ tự động dùng GPU nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang sử dụng thiết bị: {device}")

# Định nghĩa lại kiến trúc model (phải giống hệt lúc huấn luyện)
class BiLSTM_CNN_CRF(nn.Module):
    def __init__(self, vocab_size, char_vocab_size, tag_to_idx,
                 word_emb_dim, char_emb_dim, char_cnn_out_channels,
                 hidden_dim, num_lstm_layers, dropout):
        super(BiLSTM_CNN_CRF, self).__init__()
        self.tagset_size = len(tag_to_idx)
        self.word_embedding = nn.Embedding(vocab_size, word_emb_dim, padding_idx=0)
        self.char_embedding = nn.Embedding(char_vocab_size, char_emb_dim, padding_idx=0)
        self.char_cnn = nn.Conv1d(char_emb_dim, char_cnn_out_channels, kernel_size=3, padding=1)
        self.bilstm = nn.LSTM(word_emb_dim + char_cnn_out_channels, hidden_dim // 2,
                              num_layers=num_lstm_layers, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.crf = CRF(self.tagset_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, words, chars, tags=None, mask=None):
        word_embeds = self.word_embedding(words)
        batch_size, seq_len, word_len = chars.shape
        chars = chars.view(batch_size * seq_len, word_len)
        char_embeds = self.char_embedding(chars).transpose(1, 2)
        char_cnn_out = self.char_cnn(char_embeds)
        char_features = torch.max(char_cnn_out, 2)[0].view(batch_size, seq_len, -1)
        combined_embeds = self.dropout(torch.cat((word_embeds, char_features), 2))
        lstm_out, _ = self.bilstm(combined_embeds)
        emissions = self.hidden2tag(self.dropout(lstm_out))
        if tags is not None:
            return -self.crf(emissions, tags, mask=mask.byte(), reduction='mean')
        else:
            return self.crf.decode(emissions, mask=mask.byte())

# Định nghĩa hàm dự đoán
def predict_bilstm(sentence, model, word_to_idx, char_to_idx, idx_to_tag):
    model.eval()
    tokens = sentence.split()
    if not tokens: return []

    word_indices = [word_to_idx.get(w, word_to_idx["<UNK>"]) for w in tokens]
    char_indices_per_word = [[char_to_idx.get(c, char_to_idx["<UNK>"]) for c in w] for w in tokens]

    max_word_len = max([len(c) for c in char_indices_per_word if c] or [1])
    padded_chars = []
    for chars in char_indices_per_word:
        padded = torch.zeros(max_word_len, dtype=torch.long)
        if chars: padded[:len(chars)] = torch.tensor(chars)
        padded_chars.append(padded)

    words_tensor = torch.tensor([word_indices], dtype=torch.long).to(device)
    chars_tensor = torch.stack(padded_chars).unsqueeze(0).to(device)
    mask = torch.ones_like(words_tensor, dtype=torch.bool).to(device)

    with torch.no_grad():
        predicted_tag_indices = model(words_tensor, chars_tensor, mask=mask)[0]

    predicted_tags = [idx_to_tag.get(i, 'O') for i in predicted_tag_indices]

    entities, current_entity, current_tag = [], "", ""
    for token, tag in zip(tokens, predicted_tags):
        token_to_add = token.replace("_", " ")
        if tag.startswith("B-"):
            if current_entity: entities.append({"entity": current_entity.strip(), "tag": current_tag})
            current_entity, current_tag = token_to_add + " ", tag[2:]
        elif tag.startswith("I-") and current_tag and tag[2:] == current_tag:
            current_entity += token_to_add + " "
        else:
            if current_entity: entities.append({"entity": current_entity.strip(), "tag": current_tag})
            current_entity, current_tag = "", ""
    if current_entity: entities.append({"entity": current_entity.strip(), "tag": current_tag})

    return entities

# --- BƯỚC 3: TẢI MODEL VÀ TỪ ĐIỂN ---

# Chọn model bạn muốn test (syllable hoặc word)
DATA_TYPE = "word" # <-- THAY ĐỔI "syllable" hoặc "word" ở đây

MODEL_DIR = f"/content/drive/MyDrive/kpi_ner_outputs_bilstm_with_vocabs/bilstm-cnn-crf-{DATA_TYPE}"
print(f"Đang tải model và từ điển cho: {DATA_TYPE}")

# Tải từ điển từ file vocabs.json
vocab_path = os.path.join(MODEL_DIR, "vocabs.json")
try:
    with open(vocab_path, "r", encoding="utf-8") as f:
        vocabs = json.load(f)
    word_to_idx = vocabs["word_to_idx"]
    char_to_idx = vocabs["char_to_idx"]
    tag_to_idx = vocabs["tag_to_idx"]
    idx_to_tag = {int(i): t for t, i in tag_to_idx.items()}
    print("Đã tải từ điển thành công.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file 'vocabs.json' tại '{MODEL_DIR}'.")
    raise

# Khởi tạo mô hình với đúng cấu trúc từ từ điển đã tải
model_to_test = BiLSTM_CNN_CRF(
    vocab_size=len(word_to_idx), char_vocab_size=len(char_to_idx), tag_to_idx=tag_to_idx,
    word_emb_dim=100, char_emb_dim=30, char_cnn_out_channels=30, hidden_dim=256, num_lstm_layers=2, dropout=0.5
).to(device)

# Tải trọng số đã huấn luyện
model_weights_path = os.path.join(MODEL_DIR, "best_model.pt")
model_to_test.load_state_dict(torch.load(model_weights_path, map_location=device))
print(f"Đã tải trọng số model thành công.")

# Khởi tạo VnCoreNLP nếu cần test mô hình word
vncorenlp_segmenter = None
if DATA_TYPE == "word":
    print("\nKhởi tạo VnCoreNLP cho mô hình word-level...")
    # Dùng phương pháp cài đặt ổn định
    !mkdir -p vncorenlp/models/wordsegmenter
    if not os.path.exists('vncorenlp/VnCoreNLP-1.2.jar'):
        !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar -q -O vncorenlp/VnCoreNLP-1.2.jar
        !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab -q -O vncorenlp/models/wordsegmenter/vi-vocab
        !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr -q -O vncorenlp/models/wordsegmenter/wordsegmenter.rdr
    vncorenlp_segmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.2.jar", annotators="wseg", max_heap_size='-Xmx500m')
    print("VnCoreNLP sẵn sàng.")

# --- BƯỚC 4: THỰC HIỆN DỰ ĐOÁN ---

# Danh sách các câu bạn muốn test
raw_sentences_list = [
    "Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024",
    "Anh Nam cần tăng tỷ lệ phản hồi khách hàng lên 95% trong quý 3 năm nay.",
    "Chị Lan phải giảm thời gian xử lý đơn hàng xuống dưới 2 giờ trước cuối tháng này.",
    "Bộ phận Marketing cần duy trì lượng truy cập trang web ở mức 10,000 lượt/ngày trong suốt quý 2.",
    "Tăng doanh số bán hàng thêm 15% so với cùng kỳ năm ngoái.",
    "Anh Tuấn cần đạt được chỉ số NPS trên 70 điểm trong vòng 2 tháng.",
    "Mục tiêu: giảm 10% chi phí vận hành trong quý 4."
]

print("\n" + "="*20 + " BẮT ĐẦU DỰ ĐOÁN " + "="*20)
for i, sentence in enumerate(raw_sentences_list):
    print(f"\n--- Câu {i+1}: {sentence} ---")

    # Tiền xử lý câu đầu vào cho phù hợp với model
    input_sentence = ""
    if DATA_TYPE == "syllable":
        input_sentence = " ".join(sentence.split())
    elif DATA_TYPE == "word":
        if vncorenlp_segmenter:
            tokenized = vncorenlp_segmenter.tokenize(sentence)
            input_sentence = " ".join(tokenized[0])
        else:
            print("Lỗi: VnCoreNLP chưa được khởi tạo để tách từ.")
            continue

    print(f"Input cho model ({DATA_TYPE}-level): {input_sentence}")

    # Gọi hàm dự đoán
    predictions = predict_bilstm(input_sentence, model_to_test, word_to_idx, char_to_idx, idx_to_tag)

    print("Các thực thể được tìm thấy:")
    if not predictions:
        print("  (Không tìm thấy thực thể nào)")
    else:
        for entity in predictions:
            print(f"  - Thực thể: '{entity['entity']}'")
            print(f"    - Nhãn: {entity['tag']}")

print("\n" + "="*20 + " HOÀN TẤT " + "="*20)

  Cloning https://github.com/kmkurn/pytorch-crf.git to /tmp/pip-req-build-7ru2wyxb
  Running command git clone --filter=blob:none --quiet https://github.com/kmkurn/pytorch-crf.git /tmp/pip-req-build-7ru2wyxb
  Resolved https://github.com/kmkurn/pytorch-crf.git to commit 623e3402d00a2728e99d6e8486010d67c754267b
  Preparing metadata (setup.py) ... done
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Đang sử dụng thiết bị: cpu
Đang tải model và từ điển cho: word
Đã tải từ điển thành công.
Đã tải trọng số model thành công.

Khởi tạo VnCoreNLP cho mô hình word-level...
VnCoreNLP sẵn sàng.

==================== BẮT ĐẦU DỰ ĐOÁN ====================

--- Câu 1: Tăng trưởng doanh thu mảng khách hàng cá nhân đạt 20% trong quý 4 năm 2024 ---
Input cho model (word-level): Tăng_trưởng doanh_thu mảng khách_hàng cá_nhân đạt 20% trong quý 4 năm 2024
Các thực thể được tìm thấy:
  - Thực thể: 'Tăng trưởng doanh thu mảng khá